# Preprocessing steps to create dataframes

In [1]:
import pandas as pd
import csv
import numpy as np

from anomalies_analyser import compare_population_with_web
from preprocessing import create_dbpedia_raw_df, filter_raw_df, extract_populations
from dbpedia_common import DataFrameRepository, TARGET_HELD_OUT_CSV, TARGET_DEV_CSV, DBPEDIA_RAW_CSV, ALL_RELATIONS_CSV, FINAL_DBPEDIA_RAW_CSV, display_side_by_side
from dataframe_builder import DbpediaTidyDataframeBuilder
random_state = 23

In [2]:
dbpedia_raw_df = create_dbpedia_raw_df(random_state=random_state)
dbpedia_raw_df

,subject,relation,object
0,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Location>
1,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Place>
2,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/PopulatedPlace>
3,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Settlement>
4,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://schema.org/City>
...,...,...,...
2704575,<http://dbpedia.org/resource/‘Aziziya>,<http://dbpedia.org/ontology/country>,<http://dbpedia.org/resource/Libya>
2704576,<http://dbpedia.org/resource/‘Aziziya>,<http://dbpedia.org/ontology/subdivision>,<http://dbpedia.org/resource/Jafara>
2704577,<http://dbpedia.org/resource/‘Aziziya>,<http://dbpedia.org/ontology/subdivision>,<http://dbpedia.org/resource/Tripolitania>
2704578,<http://dbpedia.org/resource/‘Aziziya>,<http://dbpedia.org/ontology/timeZone>,<http://dbpedia.org/resource/Eastern_European_...


In [3]:
populations_df = extract_populations(dbpedia_raw_df)
populations_df[populations_df.estimated.notnull()].head(10)

Cities with populationTotal:  51273
Cities with populationUrban:  1871
Removed rows with less than 1000 population: 2159
Rows with Missing population: 57


relation,estimated,target
subject,,
<http://dbpedia.org/resource/A_Coruña>,0.000,nan
<http://dbpedia.org/resource/Algeciras>,0.000,nan
<http://dbpedia.org/resource/Augsburg>,0.000,nan
<http://dbpedia.org/resource/Ayagoz>,0.000,nan
<http://dbpedia.org/resource/Bacolod>,0.000,nan
<http://dbpedia.org/resource/Baguio>,0.000,nan
<http://dbpedia.org/resource/Basauri>,0.000,nan
<http://dbpedia.org/resource/Bilbao>,0.000,nan
<http://dbpedia.org/resource/Braunschweig>,0.000,nan


### Since there are only 57 cities we can scrape wikipedia with a helper function

In [4]:
error_analysis_df = populations_df[populations_df.target.isnull()].reset_index()
analysis_df = compare_population_with_web(error_analysis_df, target_col='estimated', limit=None)
analysis_df.head(10)

1 Errors found while parsing wikipedia webpage. Make a manual check for: ['Morelos,_Coahuila']
There are 57 suspicious subjects! Check if found values are real and fix the records


/home/fernando/anaconda3/envs/art_feat_eng_book/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,pretty_subject,error,expected_value,found_value,relative_log_diff,message,wiki_link,subject
42,"Morelos,_Coahuila",True,0.000,1,inf,Too small population,https://en.wikipedia.org/wiki/Morelos%2C_Coahuila,"<http://dbpedia.org/resource/Morelos,_Coahuila>"
0,A_Coruña,False,0.000,244850,inf,,https://en.wikipedia.org/wiki/A_Coru%C3%B1a,<http://dbpedia.org/resource/A_Coruña>
1,Algeciras,False,0.000,121414,inf,,https://en.wikipedia.org/wiki/Algeciras,<http://dbpedia.org/resource/Algeciras>
2,Augsburg,False,0.000,295135,inf,,https://en.wikipedia.org/wiki/Augsburg,<http://dbpedia.org/resource/Augsburg>
3,Ayagoz,False,0.000,37537,inf,,https://en.wikipedia.org/wiki/Ayagoz,<http://dbpedia.org/resource/Ayagoz>
4,Bacolod,False,0.000,561875,inf,,https://en.wikipedia.org/wiki/Bacolod,<http://dbpedia.org/resource/Bacolod>
5,Baguio,False,0.000,345366,inf,,https://en.wikipedia.org/wiki/Baguio,<http://dbpedia.org/resource/Baguio>
6,Basauri,False,0.000,40762,inf,,https://en.wikipedia.org/wiki/Basauri,<http://dbpedia.org/resource/Basauri>
7,Bilbao,False,0.000,345821,inf,,https://en.wikipedia.org/wiki/Bilbao,<http://dbpedia.org/resource/Bilbao>
8,Braunschweig,False,0.000,248292,inf,,https://en.wikipedia.org/wiki/Braunschweig,<http://dbpedia.org/resource/Braunschweig>


* ### I manually checked the rows and they are ok. Regarding Morelos,_Coahuila, it has a population of 6800
* #### I'm also adding some cities found during trainning that had errors in population column. I used cross validation and a reliable linear regression to detect anomalies and then used the same helper function to get the real value

In [5]:
fixed_populations = {
    'Morelos,_Coahuila': 6800,   
}
    
fixed_pop_df = analysis_df[['subject', 'pretty_subject']].copy()
fixed_pop_df['target'] = analysis_df[['expected_value', 'found_value']].max(axis=1)
for name, target in fixed_populations.items():
    fixed_pop_df.loc[analysis_df['pretty_subject'] == name, 'target'] = target
    
    
fixed_pop_df = fixed_pop_df.drop(columns='pretty_subject')
fixed_pop_df = fixed_pop_df.set_index('subject')

assert fixed_pop_df.target.isnull().sum() == 0
populations_df.loc[populations_df.index.isin(fixed_pop_df.index.values), 'target'] = fixed_pop_df.target
assert populations_df.target.isnull().sum() == 0


populations_df.loc['<http://dbpedia.org/resource/Lages>', 'target'] = 157544 # Found during EDA, see EDA.ipynb
populations_df.loc['<http://dbpedia.org/resource/Sarangpur,_Madhya_Pradesh>', 'target'] = 37435 # Found during EDA, see EDA.ipynb

populations_df.loc['<http://dbpedia.org/resource/Barcelona>', 'target'] = 1620343 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Buenos_Aires>', 'target'] = 2891082 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Eden_Hill,_Western_Australia>', 'target'] = 3454  # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Yuzhou,_Henan>', 'target'] = 1300000 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Erp,_Netherlands>', 'target'] = 6743 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Salinas_de_Hidalgo>', 'target'] = 16839 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Greater_Sudbury>', 'target'] = 161531 # Found during training, see linear_model.ipynb

populations_df.loc['<http://dbpedia.org/resource/Saint-Prex>', 'target'] = 5679 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Atkinson,_Nebraska>', 'target'] = 1245 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/King,_Ontario>', 'target'] = 24512 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Chirala>', 'target'] = 172826 # Found during training, see linear_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Orangeville,_Ontario>', 'target'] = 28900 # Found during training, see linear_model.ipynb

populations_df.loc['<http://dbpedia.org/resource/Maidan_Shar>', 'target'] = 14265 # Found during training, see lxgb_model.ipynb
populations_df.loc['<http://dbpedia.org/resource/Tiyeglow>', 'target'] = 2000 # Found during Error analysis

### Now we create 2 target datasets using stratification. Held out dataset for validation at the end and a train one:

In [7]:
populations_df = populations_df.reset_index()
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=random_state)
populations_df['target_category'] = pd.cut(np.floor(np.log10(populations_df["target"])),
                                           bins=[0., 3.99, 4.99, 5.99, 6.99, np.inf],
                                           labels=[3, 4, 5, 6, 7])

train_index, test_index = next(split.split(populations_df, populations_df["target_category"]))
populations_df.loc[train_index, 'test'] = False
populations_df.loc[test_index, 'test'] = True

populations_df = populations_df.drop(columns='target_category')

populations_df.loc[populations_df.test == False, ['subject', 'target']].to_csv(TARGET_DEV_CSV, index=False)
populations_df.loc[populations_df.test == True, ['subject', 'target']].to_csv(TARGET_HELD_OUT_CSV, index=False)

### Lastly, we should transform our data from a messy to a tidy format. The problem we are facing here is that there are a lot of different relations/verbs a Subject can have. But they are pretty rare, so most of them won't be useful. We have to find the common relations such as Area, Type, PlaceOfBirth?inv

#### Things to keep in mind:
* One subject can have 0, 1 or many relations of the same name. For example: Aziziya has 2 subdivisions (Jafara and Tripolitania)
* We need to distinguish relations that appear at least N% over all subjects (counting once for each subject that has that relation, no matter if one subject has it twice)
* Remove leakage features (PopulationTotal, PopulationMetro, PopulationUrban, PopulationRural)
* We need to decide how to treat multiple relations that have categorical values (ie: Aziziya has 2 subdivisions, should I create two columns? That doesn't scale)

In [8]:
%%time
cols_to_remove = ['populationTotal>', 'populationMetro>', 'populationUrban>', 'populationRural>']
ocurrence_threshold = 0.05

final_raw_df, relations_df = filter_raw_df(cols_to_remove, ocurrence_threshold)
final_raw_df

BEFORE joining populations file and raw dbpedia file:
-- Unique subjects: 77599
-- Size (rows): 2704580
AFTER joining populations file and raw dbpedia file:
-- Unique subjects: 49538
-- Size (rows): 2148819
saving relations Dataframe in input/all_relations.csv
saving final_dbpedia_raw Dataframe in input/final_dbpedia_raw.csv
AFTER removing rare relations with occurrences up to 5.0%:
-- Unique subjects: 49538
-- Unique relations: 42
CPU times: user 8.6 s, sys: 300 ms, total: 8.9 s
Wall time: 9.05 s


,subject,relation,object,target
0,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Location>,290792.000
1,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Place>,290792.000
2,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/PopulatedPlace>,290792.000
3,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Settlement>,290792.000
4,<http://dbpedia.org/resource/'Amran>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://schema.org/City>,290792.000
...,...,...,...,...
2148813,<http://dbpedia.org/resource/‘Aziziya>,<http://xmlns.com/foaf/0.1/name>,"""‘Aziziya""@en",23399.000
2148815,<http://dbpedia.org/resource/‘Aziziya>,<http://dbpedia.org/ontology/country>,<http://dbpedia.org/resource/Libya>,23399.000
2148816,<http://dbpedia.org/resource/‘Aziziya>,<http://dbpedia.org/ontology/subdivision>,<http://dbpedia.org/resource/Jafara>,23399.000
2148817,<http://dbpedia.org/resource/‘Aziziya>,<http://dbpedia.org/ontology/subdivision>,<http://dbpedia.org/resource/Tripolitania>,23399.000


### This are all the relations that appear at least 5% of the time. We'll have a lot of NaN, but we'll create one column for each of these relations to count how many of them the relation has. For example, London has +6000 <http://dbpedia.org/ontology/birthPlace?inv> relations

In [9]:
all_relations = pd.read_csv(ALL_RELATIONS_CSV)
selected_relations = all_relations.loc[all_relations.delete==False, ['relation', '%_of_occurence']]
size = len(selected_relations)
display_side_by_side(selected_relations.iloc[:int(size/2)], selected_relations[int(size/2):])

,relation,%_of_occurence
0,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,1.000
2,<http://xmlns.com/foaf/0.1/name>,0.910
3,<http://dbpedia.org/ontology/country>,0.767
4,<http://dbpedia.org/ontology/subdivision>,0.735
5,<http://dbpedia.org/ontology/areaTotal>,0.674
6,<http://dbpedia.org/ontology/postalCode>,0.648
7,<http://dbpedia.org/ontology/utcOffset>,0.628
8,<http://dbpedia.org/ontology/timeZone>,0.604
9,<http://dbpedia.org/ontology/elevation>,0.598
10,<http://dbpedia.org/ontology/areaCode>,0.588


### Now we build the training dataframe and save it

In [10]:
%%time
final_raw_df = pd.read_csv(FINAL_DBPEDIA_RAW_CSV)
relations_df = pd.read_csv(ALL_RELATIONS_CSV)
numeric_columns = [
    '<http://dbpedia.org/ontology/areaLand>',
    '<http://dbpedia.org/ontology/areaTotal>',
    '<http://dbpedia.org/ontology/areaWater>',
    '<http://dbpedia.org/ontology/elevation>',
    '<http://dbpedia.org/ontology/populationDensity>',
]

tidyDataframeBuilder = DbpediaTidyDataframeBuilder(final_raw_df, train_set=True)
(tidyDataframeBuilder
    .with_numeric_columns(numeric_columns) # Numeric columns like AreaTotal, Elevation, AreaWater and Flag columns for NaN
    .with_counter_columns() # Counter columns like areaLand#count, birthPlace?inv#count, areaCode#count
    .with_rare_relations_count(relations_df) # One column counting how many relations that have less than 5% of occurence each subject has
    .with_total_relations_count() # One column counting how many relations a subject has (including duplicates. ie: London has +6000 birthPlace?inv relations)
    .with_unique_relations_count() # One column counting how many relations a subject has, excluding duplicates.
    .with_place_types() # adds place_types column, excluding duplicates or irrelevant. ie: All subjects have "PopulatedPlace" as a place_type relation, so it's useless
    .with_offset_types() # adds offset_types column, after cleaning
    .with_countries() # adds countries column, after cleaning
)
dbpedia_df = tidyDataframeBuilder.build()

dbpedia_df.loc[dbpedia_df.subject == '<http://dbpedia.org/resource/Río_Campo>', '<http://dbpedia.org/ontology/areaLand>'] = 3300
dbpedia_df.loc[dbpedia_df.subject == '<http://dbpedia.org/resource/Río_Campo>', '<http://dbpedia.org/ontology/areaTotal>'] = 3300
dbpedia_df.loc[dbpedia_df.subject == '<http://dbpedia.org/resource/Woodstock,_Ontario>', '<http://dbpedia.org/ontology/areaLand>'] = 48575990
dbpedia_df.loc[dbpedia_df.subject == '<http://dbpedia.org/resource/Woodstock,_Ontario>', '<http://dbpedia.org/ontology/areaLand>'] = 23000000
dbpedia_df.loc[dbpedia_df.subject == '<http://dbpedia.org/resource/Woodstock,_Ontario>', '<http://dbpedia.org/ontology/populationDensity>'] = 48.960
dbpedia_df.loc[dbpedia_df.subject == '<http://dbpedia.org/resource/Tunis>', '<http://dbpedia.org/ontology/populationDensity>'] = 3004

dbpedia_df

CPU times: user 16.2 s, sys: 414 ms, total: 16.6 s
Wall time: 16.5 s


,subject,target,<http://dbpedia.org/ontology/areaLand>,<http://dbpedia.org/ontology/areaTotal>,<http://dbpedia.org/ontology/areaWater>,<http://dbpedia.org/ontology/elevation>,<http://dbpedia.org/ontology/populationDensity>,areaLand>NAN,areaTotal>NAN,areaWater>NAN,elevation>NAN,populationDensity>NAN,areaCode#count,areaLand#count,areaTotal#count,areaWater#count,birthPlace?inv#count,city?inv#count,country#count,countySeat?inv#count,deathPlace?inv#count,district#count,elevation#count,foundingDate#count,governmentType#count,gridReference#count,ground?inv#count,headquarter?inv#count,hometown?inv#count,largestCity?inv#count,licenceNumber#count,location?inv#count,motto#count,nearestCity?inv#count,originalName#count,politicalLeader#count,populationAsOf#count,populationDensity#count,postalCode#count,residence?inv#count,restingPlace?inv#count,routeEnd?inv#count,routeJunction?inv#count,routeStart?inv#count,subdivision#count,subdivision?inv#count,timeZone#count,type#count,utcOffset#count,22-rdf-syntax-ns#type#count,rdf-schema#seeAlso#count,homepage#count,name#count,nick#count,rare_rel#count,total_rel#count,unique_rel#count,place_type#cat,utc_offset#cat,country#cat
0,<http://dbpedia.org/resource/'Amran>,290792.000,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,9,0,0,1,0,2,30,10,City,3,Yemen
1,"<http://dbpedia.org/resource/'t_Zand,_Schagen>",2320.000,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,2,8,0,0,1,0,1,34,14,Town,"1,2",Netherlands
2,<http://dbpedia.org/resource/100_Mile_House>,1980.000,0.000,53290000.000,0.000,927.000,35.400,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,3,0,1,0,0,0,0,0,0,3,0,1,1,1,8,0,1,2,0,3,54,30,Town,-8,NAN
3,<http://dbpedia.org/resource/13th_district_of_...,120256.000,0.000,13430000.000,0.000,0.000,8954.000,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0,1,2,0,0,0,0,0,4,1,2,1,3,8,0,1,3,0,4,64,30,Region,"1,2",Hungary
4,<http://dbpedia.org/resource/16th_district_of_...,73486.000,0.000,33510000.000,0.000,0.000,2192.000,1,0,1,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2,1,0,1,2,0,0,0,0,0,2,1,2,1,3,8,0,1,3,0,4,64,34,Region,"1,2",Hungary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39625,"<http://dbpedia.org/resource/Əliabad,_Zaqatala>",10700.000,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,8,0,0,1,0,1,28,12,Town,"4,5",Azerbaijan
39626,"<http://dbpedia.org/resource/Əmircan,_Baku>",28203.000,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1,0,0,0,0,4,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,2,8,0,0,1,0,1,40,16,Town,"4,5",Azerbaijan
39627,<http://dbpedia.org/resource/Șoldănești>,5883.000,0.000,0.000,0.000,145.000,0.000,1,1,1,0,1,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,2,9,0,1,1,0,1,40,20,City,"2,3",Moldova
39628,"<http://dbpedia.org/resource/ʻEwa_Beach,_Hawaii>",14955.000,3625983.354,4920977.410,1294994.055,3.048,4172.700,0,0,0,0,0,1,2,2,2,2,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,0,1,1,1,6,0,0,1,0,2,54,32,NAN,-10,NAN


In [15]:
dataframe_repository = DataFrameRepository(version='last')
dataframe_repository.save(dbpedia_df)

# Columns

#### NUMERIC
* As you saw above, there are 42 of +350 relations that appears at least 5% of the time in these subjects.
* 5 of 42 are numeric columns like Area, Elevation, etc
* I added a flag column to each one of these to mark it when that instance has a NaN

In [12]:
numeric_columns = [
    '<http://dbpedia.org/ontology/areaLand>',
    '<http://dbpedia.org/ontology/areaTotal>',
    '<http://dbpedia.org/ontology/areaWater>',
    '<http://dbpedia.org/ontology/elevation>',
    '<http://dbpedia.org/ontology/populationDensity>',
    'areaLand>NAN', 'areaTotal>NAN', 'areaWater>NAN', 'elevation>NAN', 'populationDensity>NAN'
]
dbpedia_df = DataFrameRepository(version='last').get()
dbpedia_df[numeric_columns]

,<http://dbpedia.org/ontology/areaLand>,<http://dbpedia.org/ontology/areaTotal>,<http://dbpedia.org/ontology/areaWater>,<http://dbpedia.org/ontology/elevation>,<http://dbpedia.org/ontology/populationDensity>,areaLand>NAN,areaTotal>NAN,areaWater>NAN,elevation>NAN,populationDensity>NAN
0,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1
1,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1
2,0.000,53290000.000,0.000,927.000,35.400,1,0,1,0,0
3,0.000,13430000.000,0.000,0.000,8954.000,1,0,1,1,0
4,0.000,33510000.000,0.000,0.000,2192.000,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
39625,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1
39626,0.000,0.000,0.000,0.000,0.000,1,1,1,1,1
39627,0.000,0.000,0.000,145.000,0.000,1,1,1,0,1
39628,3625983.354,4920977.410,1294994.055,3.048,4172.700,0,0,0,0,0


#### CATEGORICAL
* Most of these 42 are useless per se (too many categorical values with a lot of NaN and a great number of unique values (ie: birthPlace?inv>). 
* I selected the ones with useful information (see EDA). These are place_type#cat	utc_offset#cat	country#cat
* Most of the rest had like 80% of NaN, and the values were too disperse
* Others had less than 50% of NaN, but the number of unique values were huge (name, birthPlace?inv>,  deathPlace?inv>)
* For the 3 I choose, I had to decide what to do when there were more than one value for that column, and I did a lot of cleanning
* place_type: I filer values that appeared everytime like "PopulatedPlace", and remove values that were synonims of others like "Administrative Region"
* utc_offset#cat: I decided to concat all values because a place can have more than one utc_offset
* country#cat: I choose to have just one country per subject in this category. Removing the least common ones when there where more than one country per subject. Some countries from England had this problem

In [12]:
categorical_columns = ['subject', 'place_type#cat', 'utc_offset#cat', 'country#cat']
dbpedia_df[categorical_columns]

,subject,place_type#cat,utc_offset#cat,country#cat
0,<http://dbpedia.org/resource/'Amran>,City,3,Yemen
1,"<http://dbpedia.org/resource/'t_Zand,_Schagen>",Town,"1,2",Netherlands
2,<http://dbpedia.org/resource/100_Mile_House>,Town,-8,NAN
3,<http://dbpedia.org/resource/13th_district_of_...,Region,"1,2",Hungary
4,<http://dbpedia.org/resource/16th_district_of_...,Region,"1,2",Hungary
...,...,...,...,...
39625,"<http://dbpedia.org/resource/Əliabad,_Zaqatala>",Town,"4,5",Azerbaijan
39626,"<http://dbpedia.org/resource/Əmircan,_Baku>",Town,"4,5",Azerbaijan
39627,<http://dbpedia.org/resource/Șoldănești>,City,"2,3",Moldova
39628,"<http://dbpedia.org/resource/ʻEwa_Beach,_Hawaii>",NAN,-10,NAN


### Special counters
 * total_rel#count: How many relations a subject has (counting duplicates)
 * unique_rel#count: How many unique relations a subject has
 * rare_rel#count: How many rare relations (ie: the ones that appear less than 5%) a subject has 

In [13]:
special_counter_columns = [
    'subject',
    'total_rel#count',
    'unique_rel#count',
    'rare_rel#count',
]
dbpedia_df = DataFrameRepository(version='last').get()
dbpedia_df[special_counter_columns]

,subject,total_rel#count,unique_rel#count,rare_rel#count
0,<http://dbpedia.org/resource/'Amran>,30,10,2
1,"<http://dbpedia.org/resource/'t_Zand,_Schagen>",34,14,1
2,<http://dbpedia.org/resource/100_Mile_House>,54,30,3
3,<http://dbpedia.org/resource/13th_district_of_...,64,30,4
4,<http://dbpedia.org/resource/16th_district_of_...,64,34,4
...,...,...,...,...
39625,"<http://dbpedia.org/resource/Əliabad,_Zaqatala>",28,12,1
39626,"<http://dbpedia.org/resource/Əmircan,_Baku>",40,16,1
39627,<http://dbpedia.org/resource/Șoldănești>,40,20,1
39628,"<http://dbpedia.org/resource/ʻEwa_Beach,_Hawaii>",54,32,2


### Counters
* For each one of the 42 selected relations, we have a counter per instance


In [14]:
counters_columns = dbpedia_df.columns[dbpedia_df.columns.str.endswith('#count')].values
dbpedia_df = DataFrameRepository(version='last').get()
dbpedia_df[counters_columns]

,areaCode#count,areaLand#count,areaTotal#count,areaWater#count,birthPlace?inv#count,city?inv#count,country#count,countySeat?inv#count,deathPlace?inv#count,district#count,elevation#count,foundingDate#count,governmentType#count,gridReference#count,ground?inv#count,headquarter?inv#count,hometown?inv#count,largestCity?inv#count,licenceNumber#count,location?inv#count,motto#count,nearestCity?inv#count,originalName#count,politicalLeader#count,populationAsOf#count,populationDensity#count,postalCode#count,residence?inv#count,restingPlace?inv#count,routeEnd?inv#count,routeJunction?inv#count,routeStart?inv#count,subdivision#count,subdivision?inv#count,timeZone#count,type#count,utcOffset#count,22-rdf-syntax-ns#type#count,rdf-schema#seeAlso#count,homepage#count,name#count,nick#count,rare_rel#count,total_rel#count,unique_rel#count
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,9,0,0,1,0,2,30,10
1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,2,8,0,0,1,0,1,34,14
2,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,3,0,1,0,0,0,0,0,0,3,0,1,1,1,8,0,1,2,0,3,54,30
3,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0,1,2,0,0,0,0,0,4,1,2,1,3,8,0,1,3,0,4,64,30
4,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2,1,0,1,2,0,0,0,0,0,2,1,2,1,3,8,0,1,3,0,4,64,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39625,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,8,0,0,1,0,1,28,12
39626,0,0,0,0,4,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,2,8,0,0,1,0,1,40,16
39627,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,2,9,0,1,1,0,1,40,20
39628,1,2,2,2,2,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,0,1,1,1,6,0,0,1,0,2,54,32


### Let's see Buenos Aires

In [15]:
dbpedia_df = DataFrameRepository(version='last').get()
dbpedia_df.loc[dbpedia_df.subject == '<http://dbpedia.org/resource/Buenos_Aires>', counters_columns]

,areaCode#count,areaLand#count,areaTotal#count,areaWater#count,birthPlace?inv#count,city?inv#count,country#count,countySeat?inv#count,deathPlace?inv#count,district#count,elevation#count,foundingDate#count,governmentType#count,gridReference#count,ground?inv#count,headquarter?inv#count,hometown?inv#count,largestCity?inv#count,licenceNumber#count,location?inv#count,motto#count,nearestCity?inv#count,originalName#count,politicalLeader#count,populationAsOf#count,populationDensity#count,postalCode#count,residence?inv#count,restingPlace?inv#count,routeEnd?inv#count,routeJunction?inv#count,routeStart?inv#count,subdivision#count,subdivision?inv#count,timeZone#count,type#count,utcOffset#count,22-rdf-syntax-ns#type#count,rdf-schema#seeAlso#count,homepage#count,name#count,nick#count,rare_rel#count,total_rel#count,unique_rel#count
5011,1,2,1,0,2439,62,1,0,968,0,2,2,1,0,16,147,136,0,0,270,0,0,1,2,0,0,0,38,12,0,0,1,0,52,1,2,1,9,19,0,2,2,141,8380,54
